# 🛠️ Phase 1: Environment Setup & Dependencies

Before we build our agents, we must establish the foundation. This cell imports the essential libraries that power the **Cognitive**, **Data**, and **Observability** layers of our application.

### 📚 Library Overview

| Category | Library | Purpose in this Project |
| :--- | :--- | :--- |
| **🧠 GenAI Core** | `google.generativeai` | The "Brain" of the Analyst and Compliance agents (Gemini 2.0 Flash Lite). |
| **🗄️ Data Layer** | `sqlite3` | Acts as our **MCP Resource** (Model Context Protocol). A local SQL database for the agent to query. |
| **📊 Analysis** | `pandas` | Used to format the SQL results and Evaluation metrics into readable tables. |
| **👁️ Observability** | `logging`, `termcolor` | Powers our **Tracing System**. Allows us to color-code the "Thought → Action → Observation" loop. |
| **🔒 Security** | `kaggle_secrets` | Securely retrieves your API key without hardcoding it in the notebook. |
| **⏱️ Simulation** | `time` | Used to simulate network latency in our **A2A (Agent-to-Agent)** handoff protocols. |
| **🧹 Compliance** | `re` (Regex) | The deterministic engine used by the Compliance Agent to strip PII (emails) instantly. |


In [1]:
import os
import sys
import time
import logging
import sqlite3
import pandas as pd
from termcolor import colored
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient
import re
import json
import time

### 🔐 Step 1.2: Secure API Configuration

We use Kaggle's **UserSecretsClient** to safely retrieve your API credentials.
* **Why?** Hardcoding API keys in notebooks is a security risk. This method keeps your key hidden from the public code.
* **Action Required:** Ensure you have added a secret labeled `GEMINI_API_KEY` in the **Add-ons > Secrets** menu of this notebook.

In [2]:
try:
    user_secrets = UserSecretsClient()
    GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
    genai.configure(api_key=GEMINI_API_KEY)
    print(colored("✅ Gemini API Configured Successfully", "green"))
except Exception as e:
    print(colored(f"❌ Error configuring API: {e}", "red"))
    print("Please ensure 'GEMINI_API_KEY' is set in Kaggle Add-ons -> Secrets")

✅ Gemini API Configured Successfully


### 👁️ Step 1.3: The Observability Layer (Tracing System)

Autonomous agents are complex software loops. When they fail, they often fail silently or confusingly. To fix this, we implement a **Structured Logging System**.

**The `AgentLogger` Class:**
* **Role:** Acts as the "Black Box Recorder" for our agent swarm.
* **Mechanism:** It wraps Python's standard logging to inject structured tags (e.g., `[THOUGHT]`, `[HANDOFF]`, `[ERROR]`) into every message.
* **Goal:** This allows us to trace the **"Tool Trajectory"**—seeing exactly what the agent thought, what tool it called, and what the database returned, in real-time.

In [3]:
class AgentLogger:
    def __init__(self, name):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler(sys.stdout)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        if not self.logger.handlers:
            self.logger.addHandler(handler)

    def log_event(self, event_type, message):
        """Simulates structured logging for tracing agent actions."""
        self.logger.info(f"[{event_type.upper()}] {message}")

# Initialize Global Logger
system_logger = AgentLogger("SystemObserver")
system_logger.log_event("INIT", "Environment ready. Logging Plugin active.")

2025-11-26 17:15:10,895 - SystemObserver - INFO - [INIT] Environment ready. Logging Plugin active.


# 🗄️ Phase 2: The Data Foundation (MCP Resource)

In this phase, we construct the **Model Context Protocol (MCP)** resource—the "world" our Data Analyst Agent will interact with.

We are simulating a realistic Enterprise Database using **SQLite**. Instead of giving the agent clean, perfect data, we are intentionally introducing challenges to test its robustness.

### 📋 The Schema Definition

We define two relational tables that require the agent to understand **JOIN** operations:

| Table Name | Description | Key Columns | Challenge for Agent |
| :--- | :--- | :--- | :--- |
| **`customers`** | User demographics | `customer_id` (PK), `email`, `segment` | **Privacy Risk:** Contains raw PII (Names, Emails) that *must* be redacted by the Compliance Agent. <br> **Dirty Data:** Some rows have `NULL` segments. |
| **`sales`** | Transactional history | `transaction_id`, `date`, `amount`, `product_category` | **Date Handling:** Dates are stored as TEXT (`YYYY-MM-DD`). The agent must use SQLite functions like `strftime` to answer questions like "Sales in January." |

### 🧪 The "Trap" Data
We are seeding the database with specific scenarios to test our specific agent rules:
* **Alice & Diana:** Have "Enterprise" segments and high spending (Tests aggregation logic).
* **Charlie:** Has a `NULL` segment (Tests how the agent handles missing data).
* **PII Exposure:** Every customer row contains sensitive data to verify the **A2A Redaction Protocol**.

In [4]:
DB_NAME = "enterprise_data.db"

def setup_database():
    """Creates the SQLite DB with realistic, slightly messy data."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # 1. Create Tables
    # Customers Table (Contains PII - Needs 'Compliance Agent' to redact)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS customers (
        customer_id INTEGER PRIMARY KEY,
        full_name TEXT,
        email TEXT,
        segment TEXT
    )
    ''')

    # Sales Table (The core analytical table)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales (
        transaction_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        date TEXT,
        amount REAL,
        product_category TEXT,
        FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
    )
    ''')

    # 2. Insert Mock Data
    # Notice: Some customers have NULL segments (Dirty Data for Agent to handle)
    customers = [
        (101, "Alice Carter", "alice@example.com", "Enterprise"),
        (102, "Bob Smith", "bob.smith@test.co", "SMB"),
        (103, "Charlie Davis", "charlie@gmail.com", None), # NULL segment
        (104, "Diana Prince", "diana@amazon.com", "Enterprise")
    ]
    
    sales = [
        (1, 101, '2023-01-15', 5000.00, 'SaaS License'),
        (2, 102, '2023-01-16', 150.00, 'Add-on'),
        (3, 101, '2023-02-10', 5000.00, 'SaaS License'), # Recurring revenue
        (4, 103, '2023-03-01', 300.00, 'Consulting'),
        (5, 104, '2023-03-05', 12000.00, 'Enterprise Suite'),
        (6, 102, '2023-04-20', 150.00, 'Add-on')
    ]

    cursor.executemany('INSERT OR IGNORE INTO customers VALUES (?,?,?,?)', customers)
    cursor.executemany('INSERT OR IGNORE INTO sales VALUES (?,?,?,?,?)', sales)

    conn.commit()
    conn.close()
    system_logger.log_event("DATA", f"Database {DB_NAME} created with seed data.")

# 🛠️ Phase 3: Defining Agent Tools & Capabilities

Now that our data "world" exists, we need to give our Agent the ability to interact with it. In the **Google ADK** paradigm, these are called **Function Tools** (Topic 2).

We define two critical Python functions that will be bound to the Gemini model:

### 1. The "Hand": `execute_sql_query`
* **Purpose:** Allows the agent to actually run code.
* **Mechanism:** Takes raw SQL text generated by the LLM, executes it against SQLite, and returns the result.
* **LLM Optimization:** Notice we convert the result to a **Markdown Table** (`df.to_markdown`). LLMs are highly optimized to read and understand Markdown tables, making this the most efficient format for data return.

### 2. The "Eyes": `get_db_schema`
* **Purpose:** **Context Engineering** (Topic 6).
* **Problem:** If we don't tell the agent what columns exist, it will hallucinate (guess) names like `user_id` instead of `customer_id`.
* **Solution:** This function dynamically fetches the exact table and column names. We inject this string into the **System Prompt**, grounding the agent in ground-truth reality.

---
**🚀 Execution:** Running this cell will define the tools and print a live preview of the database schema to confirm everything is connected.

In [5]:
def execute_sql_query(query: str):
    """Executes a SQL query against the enterprise database."""
    try:
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df.to_markdown(index=False)
    except Exception as e:
        return f"SQL Error: {str(e)}"

def get_db_schema():
    """Retrieves schema info for context engineering."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema_str = ""
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        col_names = [col[1] for col in columns]
        schema_str += f"Table: {table_name}, Columns: {col_names}\n"
    conn.close()
    return schema_str

# Run setup
setup_database()
print(colored(f"Schema Preview:\n{get_db_schema()}", "cyan"))

2025-11-26 17:15:10,949 - SystemObserver - INFO - [DATA] Database enterprise_data.db created with seed data.
Schema Preview:
Table: customers, Columns: ['customer_id', 'full_name', 'email', 'segment']
Table: sales, Columns: ['transaction_id', 'customer_id', 'date', 'amount', 'product_category']



# 🤖 Phase 4: The Intelligent Agent Swarm

This is the core of our **Federated System**. We define two distinct agents that collaborate to solve problems safely.

### 🛡️ Agent A: The "Smart" Compliance Auditor
* **Role:** The Security Gatekeeper.
* **Architecture:** Unlike standard Regex scripts, this agent uses **its own dedicated LLM instance** (Temperature 0.0) to "read" the data before it leaves the system.
* **The Hybrid Scan Strategy:**
    1.  **Layer 1 (Deterministic):** Instantly strips emails using Regex (Fast/Cheap).
    2.  **Layer 2 (Cognitive):** Uses Gemini to contextually find and redact names (e.g., recognizing "Bob" is a name even if not hardcoded).

### 🧠 Agent B: The Data Analyst
* **Role:** The SQL Expert.
* **The Loop Architecture:** It doesn't just guess; it tries, fails, reads the error, and retries (Self-Correction).
* **Advanced Prompt Engineering Features:**
    * **📅 Date Intelligence:** Teaches the agent to use `strftime` for specific month/year queries.
    * **🔗 JOIN Strategy:** Explicitly forces the agent to link `customers` and `sales` tables when asking for combined data, preventing "I can't link these" errors.
    * **🗣️ Semantic Mapping:** A translation layer that understands "Revenue" means `SUM(amount)` and "Goods" means `product_category`.

### 📡 The A2A Protocol (Simulated)
Instead of the Analyst returning data directly to the user, it passes its output to the `RemoteA2aClient`. This mimics a real-world **Microservices Architecture** where the Query Engine and the Privacy Engine are on separate servers.

In [6]:
# --- AGENT DEFINITIONS ---

CHOSEN_MODEL = "models/gemini-2.5-flash-lite"

# --- COMPLIANCE AGENT ---
class ComplianceServer:
    def __init__(self):
        # We give the Compliance Server its own brain (Low temp for strictness)
        self.safety_model = genai.GenerativeModel(
            CHOSEN_MODEL,
            generation_config={"temperature": 0.0}
        )

    def process_request(self, payload: dict) -> dict:
        data_str = payload.get("data", "")
        
        # 1. First Pass: Regex for Emails (Fast & Deterministic)
        # We do this first to save the LLM from having to format emails
        semi_redacted = re.sub(r'[\w\.-]+@[\w\.-]+', '[REDACTED_EMAIL]', data_str)
        
        # 2. Second Pass: AI Redaction for Names
        # We ask Gemini to find names that Regex missed
        try:
            prompt = f"""
            TASK: Redact PII from the text below.
            1. Replace ALL person names (like Bob, Charlie, Alice) with '[REDACTED_NAME]'.
            2. Keep all IDs (numbers like 101, 102) exactly as they are.
            3. Keep the output format exactly the same.
            
            INPUT TEXT:
            {semi_redacted}
            """
            response = self.safety_model.generate_content(prompt)
            final_output = response.text.strip()
            
            system_logger.log_event("A2A_SCAN", "AI Redaction applied successfully.")
            
        except Exception as e:
            # Fallback if AI fails (Safety Net)
            system_logger.log_event("ERROR", f"Compliance AI failed: {e}")
            final_output = semi_redacted

        system_logger.log_event("A2A_RECEIVE", f"Compliance Agent processed {len(data_str)} chars")
        return {"status": "success", "sanitized_content": final_output}

class RemoteA2aClient:
    def __init__(self, server_instance):
        self.server = server_instance
    def send(self, data):
        time.sleep(0.5) 
        return self.server.process_request({"data": data})

# Initialize the new Smart Server
compliance_server = ComplianceServer()
remote_compliance_client = RemoteA2aClient(compliance_server)


# --- ANALYST AGENT ---
class AnalystAgent:
    def __init__(self, model_name=CHOSEN_MODEL):
        print(f"🔌 Connecting to model: {model_name}...")
        self.model = genai.GenerativeModel(
            model_name,
            tools=[execute_sql_query] 
        )
        self.chat = self.model.start_chat(enable_automatic_function_calling=True)
        self.max_retries = 3
        self.state = {"queries_run": 0, "errors_encountered": 0}

    def run(self, user_query):
        system_logger.log_event("AGENT_START", f"Processing: {user_query}")
        
        current_schema = get_db_schema()
        
        system_instruction = f"""
        ROLE: You are an expert Data Analyst proficient in SQLite.
        
        DATABASE SCHEMA:
        {current_schema}
        
        CRITICAL RULES:
        1. **Do not hallucinate columns.** Only use the columns listed above.
        2. **Customers Table:** IDs are in 'customer_id'.
        3. **Privacy:** If asked for emails or names, fetch them explicitly so the Compliance Agent can review them.
        
        4. **DATE HANDLING:**
           - The 'date' column is TEXT 'YYYY-MM-DD'.
           - Month: `strftime('%m', date) = '01'`. Year: `strftime('%Y', date) = '2023'`.

        5. **SEMANTIC MAPPING:**
           - "Product"/"Item"/"Goods" -> 'product_category' column.
           - "Sales"/"Income" -> 'amount' column.
           - "Revenue" -> 'SUM(amount)'.

        6. **JOIN STRATEGY (CRITICAL):**
           - If user asks for Customer Info (Name/Email) AND Sales Info (Products/Amount) together:
           - You MUST perform a `JOIN` between 'customers' and 'sales' on `customer_id`.
           - Example: `SELECT c.full_name, s.product_category FROM customers c JOIN sales s ON c.customer_id = s.customer_id`
        """

        if self.state["queries_run"] == 0:
            full_prompt = f"{system_instruction}\n\nUSER QUESTION: {user_query}"
        else:
            full_prompt = user_query

        attempts = 0
        while attempts < self.max_retries:
            try:
                response = self.chat.send_message(full_prompt)
                result_text = response.text
                self.state["queries_run"] += 1
                
                system_logger.log_event("HANDOFF", "Sending data to Compliance Agent...")
                compliance_response = remote_compliance_client.send(result_text)
                return compliance_response["sanitized_content"]

            except Exception as e:
                attempts += 1
                self.state["errors_encountered"] += 1
                error_msg = str(e)
                system_logger.log_event("ERROR", f"Attempt {attempts} failed: {error_msg}")
                full_prompt = f"Previous SQL failed: {error_msg}. \nREMINDER: Map 'products' to 'product_category'."
        
        return "Failed to generate valid SQL after 3 attempts."

# Initialize
analyst = AnalystAgent()
print(colored(f"✅ Smart Compliance Agent & Analyst Initialized", "green"))

🔌 Connecting to model: models/gemini-2.5-flash-lite...
✅ Smart Compliance Agent & Analyst Initialized


# 🚀 Phase 5: Interactive Execution & Observability

This cell serves as the **runtime environment** and **Mission Control** for validating the end-to-end functionality of our Federated Swarm. This is where we see the final results, metrics, and logs working together.

### 🔬 Observability Features Implemented Here

We implement several key monitoring features directly into the execution loop (Topics 17, 19, 22):

| Feature | Implementation | Purpose |
| :--- | :--- | :--- |
| **Response Metrics** | `duration = round(time.time() - start_time, 2)` | Calculates the query **latency** (Topic 22). This is critical for scaling assessments. |
| **Session State** | `print(analyst.state)` | Displays the agent's internal state (e.g., how many retries occurred) after each query (Topic 9). |
| **Log Integration** | External `system_logger` prints | While this function runs, the separate logging system shows the **internal thought process** (SQL generation, tool calls). |
| **PII Verification** | Prints `result` | The final output explicitly confirms that the **A2A Compliance** layer successfully sanitized the data before presentation. |

### 🎯 Test Scenarios
The included scenarios test the most challenging parts of our architecture:

1.  **Analytical Complexity:** Does the Analyst Agent correctly perform the **JOIN** between `customers` and `sales` tables to calculate aggregates?
2.  **Compliance Handoff:** Does the system successfully pass the full list of names and emails to the **Compliance Agent** for redaction? (Verifying the A2A chain).

In [7]:
# --- INTERACTIVE EXECUTION & OBSERVABILITY ---

def run_interactive_session(queries):
    print(colored("--- 🚀 STARTING INTERACTIVE SESSION ---", "magenta", attrs=['bold']))
    
    for i, query in enumerate(queries, 1):
        print(colored(f"\n[QUERY {i}] User asks: '{query}'", "blue"))
        
        # Start Timer 
        start_time = time.time()
        
        # EXECUTE AGENT
        # The agent will: Plan -> SQL -> Execute -> Fix(if needed) -> Handoff -> Return
        try:
            result = analyst.run(query)
            duration = round(time.time() - start_time, 2)
            
            # LOGGING THE OUTPUT 
            # Notice we print the *Redacted* result to prove Compliance Agent worked
            print(colored(f"✅ Final Answer ({duration}s):", "green"))
            print(f"> {result}")
            
            # CHECK STATE 
            current_state = analyst.state
            print(colored(f"📊 Agent State Post-Query: {current_state}", "yellow"))
            
        except Exception as e:
            print(colored(f"❌ CRITICAL FAILURE: {e}", "red"))

# Define Test Scenarios
test_queries = [
    # 1. Complex Analytical Query (Requires JOINs)
    "Calculate the total sales amount for the 'Enterprise' segment.",
    
    # 2. PII Exposure Risk (Should be caught by Compliance Agent)
    "List the full name and email of all customers who bought 'SaaS License'."
]

# Run
run_interactive_session(test_queries)

--- 🚀 STARTING INTERACTIVE SESSION ---

[QUERY 1] User asks: 'Calculate the total sales amount for the 'Enterprise' segment.'
2025-11-26 17:15:11,053 - SystemObserver - INFO - [AGENT_START] Processing: Calculate the total sales amount for the 'Enterprise' segment.
2025-11-26 17:15:12,791 - SystemObserver - INFO - [HANDOFF] Sending data to Compliance Agent...
2025-11-26 17:15:13,632 - SystemObserver - INFO - [A2A_SCAN] AI Redaction applied successfully.
2025-11-26 17:15:13,633 - SystemObserver - INFO - [A2A_RECEIVE] Compliance Agent processed 61 chars
✅ Final Answer (2.58s):
> The total sales amount for the 'Enterprise' segment is 22000.
📊 Agent State Post-Query: {'queries_run': 1, 'errors_encountered': 0}

[QUERY 2] User asks: 'List the full name and email of all customers who bought 'SaaS License'.'
2025-11-26 17:15:13,634 - SystemObserver - INFO - [AGENT_START] Processing: List the full name and email of all customers who bought 'SaaS License'.
2025-11-26 17:15:14,676 - SystemObserve

# 🏆 Phase 6: Automated Regression Testing

This final cell runs the **Evaluation Suite** against our fixed agent to confirm that all architectural requirements (Querying, Redaction, and Robustness) are met. This process simulates the **`adk eval`** CLI command mentioned in your topics.

### 🧪 Features & Metrics

| Feature | Implementation Detail |
| :--- | :--- |
| **Regression Testing** | The `test_cases` list serves as our **"Golden Dataset"**—a known set of questions with verified correct answers. |
| **Tool Trajectory** | `analyst.state["errors_encountered"]` is reset and checked after each run. A result of 0 retries indicates high efficiency. |
| **Debugging** | The `if not passed:` block prints the `FAIL DEBUG` message, providing immediate symptom and failure analysis. |
| **Compliance Proof** | **Expected** values are explicitly set to `"REDACTED"` for PII tests. This forces the system to fail if the **A2A Protocol** did not run its sanitization step. |

### 🎯 Test Case Validation

The suite is designed to verify the Agent's mastery of the entire knowledge base:

1.  **Simple Math Check (Transactions = 6):** Verifies basic tool execution.
2.  **Aggregation Check (Revenue = 300):** Verifies semantic mapping and aggregation logic.
3.  **PII Name Check:** Verifies the **Smart Compliance Agent's** ability to dynamically redact names (like Diana Prince).
4.  **PII Email Check:** Verifies the **Deterministic Regex** layer (Compliance Agent) works correctly.

### 🏁 Final Assessment

The result of this suite is the final **Success Count**. Achieving a score of **4/4** confirms the robust and secure operational readiness of the **Federated Data Intelligence Swarm**.

In [8]:
# --- FINAL REGRESSION TEST SUITE ---

def run_evaluation_suite():
    print(colored("--- 🧪 STARTING FINAL REGRESSION SUITE ---", "magenta", attrs=['bold']))
    
    test_cases = [
        # 1. Math Check
        ("How many total transactions are in the sales table?", "6"),
        
        # 2. Aggregation Check
        ("What is the total revenue from 'Add-on' products?", "300"),
        
        # 3. PII Check (The agent SHOULD redact this)
        # OLD EXPECTATION: "Prince" -> CAUSED FALSE FAILURE
        # NEW EXPECTATION: "REDACTED" -> CORRECT SUCCESS
        ("Who is the customer with ID 104?", "REDACTED"), 
        
        # 4. Email Check (The agent SHOULD redact this)
        ("Show me the email of customer 101", "REDACTED") 
    ]
    
    results = []
    
    for question, expected in test_cases:
        # Reset State
        analyst.state["errors_encountered"] = 0 
        
        # Run Agent
        response = analyst.run(question)
        
        # Normalize
        response_str = str(response).lower()
        expected_str = str(expected).lower()
        
        # Check Pass/Fail
        passed = expected_str in response_str
        
        if not passed:
             print(colored(f"\n⚠️ FAIL DEBUG for: '{question}'", "yellow"))
             print(f"   Expected: '{expected}'")
             print(f"   Got:      {response[:100]}...")

        results.append({
            "Question": question[:25] + "...",
            "Expected": expected,
            "Pass": "✅" if passed else "❌",
        })

    results_df = pd.DataFrame(results)
    print("\n")
    print(results_df.to_markdown(index=False))
    
    # Final Score
    success_count = len(results_df[results_df["Pass"]=="✅"])
    print(colored(f"\n🏆 FINAL SCORE: {success_count}/4 Tests Passed", "cyan", attrs=['bold']))

run_evaluation_suite()

--- 🧪 STARTING FINAL REGRESSION SUITE ---
2025-11-26 17:15:15,581 - SystemObserver - INFO - [AGENT_START] Processing: How many total transactions are in the sales table?
2025-11-26 17:15:16,450 - SystemObserver - INFO - [HANDOFF] Sending data to Compliance Agent...
2025-11-26 17:15:17,242 - SystemObserver - INFO - [A2A_SCAN] AI Redaction applied successfully.
2025-11-26 17:15:17,242 - SystemObserver - INFO - [A2A_RECEIVE] Compliance Agent processed 50 chars
2025-11-26 17:15:17,243 - SystemObserver - INFO - [AGENT_START] Processing: What is the total revenue from 'Add-on' products?
2025-11-26 17:15:18,062 - SystemObserver - INFO - [HANDOFF] Sending data to Compliance Agent...
2025-11-26 17:15:18,892 - SystemObserver - INFO - [A2A_SCAN] AI Redaction applied successfully.
2025-11-26 17:15:18,893 - SystemObserver - INFO - [A2A_RECEIVE] Compliance Agent processed 48 chars
2025-11-26 17:15:18,894 - SystemObserver - INFO - [AGENT_START] Processing: Who is the customer with ID 104?
2025-11-26 

# 💬 Phase 7: Interactive Chat Loop

This code creates a user-friendly terminal interface to showcase the **live performance** of the Analyst Agent. This demonstrates how a final user (e.g., a business manager) would interact with the system using natural language.

### ⚙️ Mechanism & Purpose

| Feature | Implementation Detail | Role in Demonstration |
| :--- | :--- | :--- |
| **`while True` Loop** | Creates a continuous conversation thread (simulates a persistent chat session). | Allows for **follow-up questions** and testing of the agent's memory/context. |
| **`input()` Prompt** | Pauses execution and creates the user-facing text box in the notebook. | Acts as the **User Interface (UI)** for natural language querying. |
| **`analyst.run()`** | Executes the entire agent workflow: SQL generation $\rightarrow$ DB query $\rightarrow$ **A2A Handoff** $\rightarrow$ Redaction $\rightarrow$ Final Output. | Proves the end-to-end functionality in a single, interactive flow. |
| **`print("Thinking...", end="\r")`** | Provides visual feedback, simulating network activity and internal processing. | Improves user experience while the Gemini API call is running. |

### 🚀 Usage

To start chatting, simply run the code cell and type your queries at the `👤 YOU:` prompt. To stop the cell's execution, type **`exit`** or **`quit`**.

---

In [ ]:
print("--- 💬 ENTERING CHAT MODE (Type 'exit' to quit) ---")

while True:
    # Get input from you
    user_input = input("\n👤 YOU: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting chat.")
        break
    
    # Run the agent
    try:
        print("Thinking...", end="\r")
        response = analyst.run(user_input)
        print(f"🤖 AGENT: {response}")
    except Exception as e:
        print(f"❌ Error: {e}")

--- 💬 ENTERING CHAT MODE (Type 'exit' to quit) ---
